# Introducing OpenManipulator-X
Wir werden hier die Grundlegenden Funktionalitäten des OpenManipulator-X Roboterarms kennenlernen. Normalerweiße gibt es bereits eine Implementierung für die Vorwärts und Rückwärtskinemtic des Openmanipulator, der Roboter wird in der Gazebo-Umgebung simuliert und die Bewegung des Roboters mit Hilfe von ROS angesteuert. 

In den Übungen werden wir allerdings solbst die Kinematik implementieren und den Roberarm mithilfe von `MeshCatMechanisms.jl` und `RigidBodyDynamics.jl` simulieren und steuern.

## Installation

Zum start müssen wir noch die folgenden Pakete laden und installieren. Die Installation der Pakete kann einige Zeit in Anspruch nehmen. Ist müsst sie aber nur einmal durchführen.

> Vergesst nicht, dass ihr den Julia-Kernel für dieses Notebook auswählt. Er sollte in ```/snap/bin/julia``` sein

In [1]:
import Pkg
# Pkg.generate("OpenMEnv") # generate a new package
Pkg.activate("OpenMEnv") # activate the package
Pkg.add("RigidBodyDynamics")
Pkg.add("MeshCatMechanisms")
Pkg.add("MeshCat")
Pkg.add("LinearAlgebra")
Pkg.add("StaticArrays")

  Activating new project at `~/Library/Mobile Documents/com~apple~CloudDocs/Arbeit/Lectures/Vorlesung - Ingenieurwissenschaftliche Grundlagen 3 (Numerische Methoden)/auxme_numerik/Excercise/Exercise 3/OpenMEnv`
   Resolving package versions...
    Updating `~/Library/Mobile Documents/com~apple~CloudDocs/Arbeit/Lectures/Vorlesung - Ingenieurwissenschaftliche Grundlagen 3 (Numerische Methoden)/auxme_numerik/Excercise/Exercise 3/OpenMEnv/Project.toml`
  [366cf18f] + RigidBodyDynamics v2.4.0
    Updating `~/Library/Mobile Documents/com~apple~CloudDocs/Arbeit/Lectures/Vorlesung - Ingenieurwissenschaftliche Grundlagen 3 (Numerische Methoden)/auxme_numerik/Excercise/Exercise 3/OpenMEnv/Manifest.toml`
  [49dc2e85] + Calculus v0.5.1
  [ffbed154] + DocStringExtensions v0.9.3
  [fa6b7ba4] + DualNumbers v0.6.8
  [92d709cd] + IrrationalConstants v0.2.2
  [692b3bcd] + JLLWrappers v1.5.0
  [9c8b4983] + LightXML v0.9.1
  [2ab3a3ac] + LogExpFunctions v0.3.27
  [39f5be34] + LoopThrottle v0.1.0
  [77ba44

## Environment Initialisierung

Als nächstes müssen wir das Environment initialisieren. Dazu müssen wir die folgenden Zeilen ausführen. 

In [2]:
import Pkg
Pkg.activate("OpenMEnv") 
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using StaticArrays

  Activating project at `~/Library/Mobile Documents/com~apple~CloudDocs/Arbeit/Lectures/Vorlesung - Ingenieurwissenschaftliche Grundlagen 3 (Numerische Methoden)/auxme_numerik/Excercise/Exercise 3/OpenMEnv`
┌ Warning: attempting to remove probably stale pidfile
│   path = /Users/damian/.jlassetregistry.lock
└ @ Pidfile /Users/damian/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260


## Laden der OpenManipulator-X Beschreibung

Unter `../open_manipulator_description/urdf/` findet ihr die URDF-Datei des OpenManipulator-X. In ihr wird die Struktur des Roboters anhand von Links, Gelenken und deren Eigenschaften beschrieben. Dort sind zu dem die meshes der einzelnen Teile referenziert. Die Datei ist nicht sehr groß und übersichtlich. Schaut euch die Datei an und vergleicht sie mit der Beschreibung des Roboterarms [hardware-specification](https://emanual.robotis.com/docs/en/platform/openmanipulator_x/specification/#hardware-specification). Die URDF-Datei wird in den folgenden Schritten geladen und zur visualisierung verwendet. 

In [3]:
srcdir = "../open_manipulator_description/urdf/"
urdf = joinpath(srcdir, "open_manipulator.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: link2, Edge: joint1
    Vertex: link3, Edge: joint2
      Vertex: link4, Edge: joint3
        Vertex: link5, Edge: joint4
          Vertex: gripper_link, Edge: gripper
          Vertex: gripper_link_sub, Edge: gripper_sub
No non-tree joints.

Wie ihr seht besteht das `mechanism`objekt des Roboterarms aus einem `tree` mit `vertices` und `edges`. Vergleicht die Ausgabe mit dem folgenden Bild welches den Roboterarm zeigt. Die Beschreibung started mit dem `world` Koordinatensystem und geht anschließend von unten nach oben die `links` und `joints` entlang. In dem Bild werden für die `joints` die IDs [ID11, ID12, ID13, ID14, ID15] verwendet. Wir können also recht einfach die Struktur des Roboters sehen.

![Image](./OpenManipulator.png)

## Visualisierung

Als nächstes starten wir die Simulation und Visualisierung des Roboters. Dazu müssen wir die folgenden Zeilen ausführen.

In [4]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8707
└ @ MeshCat /Users/damian/.julia/packages/MeshCat/oC0sL/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("link1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x32, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x33, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link5" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0

Ihr könnt in der Visualisierung mit der Maus navigieren um den Roboterarm von verschiedenen Perspektiven zu betrachten. 

Als nächstes wollen wir den Roboterarm bewegen. Ohne die Kinematik zu implementieren können wir nur die Gelenkwinkel des Roboters ändern. Mit der methode `set_configuration` können wir die Gelenkwinkel des Roboters ändern. Hier können wir 7 Werte anpassen um die Gelenkwinkel zu ändern. Die ersten 4 sind die Winkle der Gelenke von unten nach oben [ID11, ID12, ID13, ID14], der 5. und 6. Wert ist für die Steuerung des Grippers. 

In [5]:
set_configuration!(mvis, [0.0, 1.5, -1.5, 0.0, 0.0, 0.0])

Wenn wir die Winkel ändern, können wir sehen wie sich der Zustand des Roboterarms verändert.

Um den Roboterarm in die Ausgangsposition zu bringen, können wir die folgenden Werte verwenden: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0].

In [6]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

Um den aktuellen Zustand des Roboters zu erhalten, können wir den state der Simulation abfragen.

In [7]:
state = mvis.state
joint_state = mvis.state.q

6-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Wir können auch die Geschwindigkeit der Gelenke abfragen. Diese werden wir aber auch noch in einer späteren Übung selbst berechnen.

In [8]:
mvis.state.v

6-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# Motivation zur Kinematik

Was ist wenn wir jetzt aber wissen wollen wo sich der Endeffektor des Roboters befindet, wenn wir die Gelenkwinkel kennen? Der Roboter hat ja ein definiertes Koordinatensystem anhand dessen wir die Tranformationen ausrechnen können. Deshalb bekommen wir alle informationen die wir benötigen aus der Simulation. Allerdings müssen wir dann die Kinematik des Roboters berechnen. Als kleiner vorgeschmack schauen wir uns noch ein paar Funktionen aus `RigidBodyDynamics` wie zum Beispiel die `transform_to_root` Methode an. Wir werden diese Methoden später selber implementieren aber um zu sehen warum wir sie brauchen, schauen wir uns sie jetzt an.

Die `transform_to_root` Methode gibt uns die Transformation eines Punktes relativ zum Basis-Koordinatensystem des Roboters an. 

In [9]:
gripper_joint = findjoint(mechanism, "gripper")
transform_to_root(mvis.state, frame_after(gripper_joint))

Transform3D from "after_gripper" to "world":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.2417, 0.021, 0.20450000000000002]

Mit der Methode `transform` können wir die Transformation eines Punktes relativ zu einem anderen Punkt berechnen. So können wir zum Beispiel die Transformation des Endeffektors relativ zum Ursprung berechnen. 

In [10]:
zero_v = zero(SVector{3})
world = findbody(mechanism, "world")

ee = transform(mvis.state, Point3D(frame_after(gripper_joint), zero_v), default_frame(world))

Point3D in "world": [0.2417, 0.021, 0.20450000000000002]

Testet die Methode mit verschiedenen Punkten und schaut ob die Transformationen korrekt sind. In der nächsten Übung werden wir die Kinematic dann selbst implementieren.

### Simulation?

Ihr habt vielleicht bemerkt, dass die Simulation nicht wirklich realistisch ist. Der Roboterarm bewegt sich nicht "wirklich". Wir setzen nur den Zustand und die Visualisierung wird aktualisiert. Um eine Bewegung zu simulieren können wir die `simulate` Methode verwenden. In dieser können wir den Zustand und eine Zeit angeben. Eine Simulation wird dann in der angegebenen Zeit durchgeführt.

Als erstes setzen wir den Roboterarm in die Ausgangsposition. Anschließend führen wir eine Simulation für 0.5 Sekunden durch. 

In [11]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [12]:
final_time = 3.0
ts, qs, vs = simulate(mvis.state, final_time);

In [13]:
animate(mvis, ts, qs);

Was passiert hier? Könnt ihr euch erklären warum der Roboterarm sich so bewegt?

Wir setzen den Roboterarm noch mal zurück und implementieren eine Methode welche die Drehmomente der Gelenke aktiv auf 0.0 setzt und führen anschließend noch mal eine Simulation durch. 

In [14]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [15]:
om_joints = joints(mechanism)
function simple_control!(torques::AbstractVector, t, state::MechanismState)
    for joint in om_joints
        torques[velocity_range(state, joint)] .= 0.0
    end
end;

In [16]:
final_time = 3.0
ts, qs, vs = simulate(mvis.state, final_time, simple_control!; Δt=1e-2);

In [17]:
animate(mvis, ts, qs);

Da wir jetzt wissen dass keine Drehmomente anliegen, sehen wir, dass die Bewegung durch die Simulation der Massen und Trägheitsmomente der Gelenke zustande kommt. Normalerweiße können wir die Servos des Roboterarms steuern und die Gelenkwinkel die wir haben wollen direkt setzen. Da aber die Definition der Gelenke als `revolute` Gelenke in der URDF-Datei definiert sind, können wir die Gelenke nur über die Drehmomente steuern. 

Als kleinen "tirck" können wir die Geschwindigkeiten der Gelenke auf 0.0 setzen und die Simulation durchführen. Wenn wir das für alle gelenke bis auf das erste Gelenk machen, können wir den Roboterarm an diesem Gelenk steuern. Zum Beispiel können wir so lange das erste Gelenk steuern bis der Roboterarm in der gewünschten Position ist.

In [18]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [19]:
function joint_position_control(q_state, v_state, q_desired)
    kp = 2.0
    v_damping = 0.5
    max_torque = 1.0

    torque = kp * (q_desired - q_state) - v_damping * v_state
    torque = clamp.(torque, -max_torque, max_torque)
    
    return torque
end;

In [20]:
function simple_joint_control!(torques::AbstractVector, t, state::MechanismState)
    joint_number = 1
    joint_position = 0.5

    for j_iter in 1:6
        if j_iter != joint_number
            state.v[j_iter] = 0.0
        end
    end
    for joint in om_joints
        torques[velocity_range(state, joint)] .= 0.0
    end
    torques[velocity_range(state, om_joints[joint_number])] .= joint_position_control(state.q[joint_number], state.v[joint_number], joint_position)
end;

In [21]:
final_time = 5.0
ts, qs, vs = simulate(mvis.state, final_time, simple_joint_control!; Δt=1e-2);
animate(mvis, ts, qs);
joint_state = mvis.state.q

6-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
 0.500020859968928
 0.0
 0.0
 0.0
 0.0
 0.0

**Aufgabe:** Testet Verschiedene Werte für Regelung der Gelenke (`kp` und `v_damping`) und schaut wie sich der Roboterarm bewegt. Was passiert bei hohen und was bei niedrigen Werten? Könnt ihr die Regelung beschreiben und wie könnte man die Bewegung des Roboterarms verbessern? 

# Zusammenfassung
Wir haben gesehen wie man den Robotarm Visualisieren und in der Simulation ansteuern kann. Wir haben auch gesehen dass es recht schwierig ist den Roboterarm zu steuern wenn wir die Positionen und die Geschwindigkeit nicht kennen und die Gelenke nur über die Drehmomente steuern können. In den nächsten Übungen werden wir uns deshalb genau diese Probleme anschauen.